<b>Privacy Preserverd LBS Advertisement Using Private Proximity Testing With Homomorphic Encryption</b><br>
1.Alice wil send her encrypted zip and category to merchants.<br>
2.Merchants will encrypt and blind their zip code and category and encrypt the advertisement back to Alice.<br>
3.Alice will check if the merchant and Alice have same Zipcode and category.<br>
4.Neither Alice nor merchants knows the other parties zip or category and only knows if they are equal with her.<br>
5.Proximity is computed using XNOR function and will be 1 if they are equal and 0 if they are not .<br>
6.Alice multiply proximity (Either 0 or 1) with encrypted cipher text advertisement.<br>
7.Alice takes only merchants with same zip and category and discard rest of the merchants.<br>
8.Ciper text will be 0 for mismatched zip or  category afetr multiplication.<br>
9.Alice decrypt the advertisement and display in her UI or mobile application. <br><br>

<b>No Party knows the whereabouts of the other parties in the transaction. </b>

In [1]:
#Elgamal Encryption
import random
import hashlib
from Crypto.Util.number import inverse
import random
import sympy
from binascii import hexlify, unhexlify

# Convert string to integer representation
def string_to_int(string):
    return int.from_bytes(string.encode(), byteorder='big')
# Convert integer representation to string
def int_to_string(integer):
    return integer.to_bytes((integer.bit_length() + 7) // 8, byteorder='big').decode()
def xnor(a, b):
    return int(a == b)

# generate large prime number p
def generate_prime(bits):
    while True:
        p = random.getrandbits(bits)
        if is_prime(p):
            return p

# check if a number is prime
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

# find a generator for the prime number p
def find_generator(p):
    for g in range(2, p):
        if pow(g, (p-1)//2, p) != 1 and pow(g, p-1, p) == 1:
            return g
    return None

def generate_keysSeperate():
    p1 =None
    while  p1 is None:
        p1 =sympy.randprime(random.getrandbits(500)*2,random.getrandbits(500)*4)# generate_prime(55)#(50)# generate_prime(1024)
    
    g1 =   find_generator(p1)
    x1 =  random.randint(1, p1-2)
    y1 =  pow(g1, x1, p1)
    return (p1, g1, y1), (p1, x1)

def generate_dynamicKeys(input_message):
    inputbytes = str.encode(input_message)
    m = int(hexlify(inputbytes), 16)
    p = sympy.randprime(m*2, m*4)
    g = sympy.randprime(int(m/2), m)
    x = random.randint(int(m/2),m)
    r = random.randint(int(m/2),m)
    y =  pow(g, x, p)
    return (p, g, y), (p, x)


# generate public and private keys
def generate_keys():
    p = generate_prime(50)# generate_prime(1024)
    g =   find_generator(p)
    x =  random.randint(1, p-2)
    y =  pow(g, x, p)
    return (p, g, y), (p, x)

# encrypt a message
def encrypt(public_key, message):
    p, g, y = public_key
    k = random.randint(1, p-2)
    r = pow(g, k, p)
    c = (message * pow(y, k, p)) % p
    return (r, c)

def decrypt (private_key, ciphertext):#(x,c1,c2,p):
    p, x = private_key
    c1, c2 = ciphertext
    s = pow(c1,x,p)
    dm = (c2 * inverse(s,p)) % p
    return dm


In [2]:
# generate public and private keys
Alice_public_key, Alice_private_key = generate_keysSeperate()


In [3]:
#Private Proximity detection using homomorphic encryption
def CalculateProximity(Alice_public_key,Alice_private_key,Alicevalue,BobValue):
    #Alice Send Value
    p, g, h =Alice_public_key  #(p, g, y), 
    p1, x= Alice_private_key   #(p, x)    
    r=random.randint(1, p-2)
    C1a= pow(g,r,p)
    C2a= pow(h, (Alicevalue+r),p) 
    
    #Bob Append And Send Value
    s=random.randint(1, p-2)
    t=random.randint(1, p-2)
    C1b= (pow(C1a, s  ,p) * pow(g,t,p)) %p 
    C2b= (pow(C2a, s  ,p) * pow(h,(t-s*BobValue),p)) %p
    
    #Alice Calculate Proximity
    p1, x= Alice_private_key 
    m=C2b/ pow(C1b,x,p)
    #m=m-1
    return m

In [4]:
Merchants = [
    (78717, 4, "McDonald's-World's best Burger"),
    (78717, 1, "Taco Bell-Best Food"),
    (78717, 2, "Walmart-Price at lowest"),
    (78717, 9, "Sushi Bar-Best Sushi In Town!"),
    (90210, 1, "Chipotle-Mexican Grill"),
    (10001, 2, "Target-Expect More, Pay Less"),
    (60611, 3, "Jiffy Lube-Leave Worry Behind"),
    (94103, 4, "AMC Theatres-Amazing Cinematic Experience"),
    (10019, 1, "Shake Shack-Modern Day Roadside Burger Stand"),
    (90046, 9, "Nobu-Malibu, CA"),
    (90210, 5, "Louis Vuitton-Timeless Elegance"),
    (10011, 6, "Apple-Think Different"),
    (60654, 7, "Lululemon-Yoga-inspired athletic wear"),
    (94123, 8, "Whole Foods Market-America's Healthiest Grocery Store"),
    (78717, 1, "Chick-fil-A-Eat Mor Chikin"),
    (78702, 2, "Best Buy-Expert Service. Unbeatable Price."),
    (78703, 3, "Firestone Complete Auto Care-Keep Your Car Running Newer, Longer"),
    (78704, 4, "Alamo Drafthouse Cinema-Experience the Difference"),
    (78705, 1, "Subway-Fresh Fit For Kids"),
    (78706, 9, "PF Chang's-China Bistro"),
    (78707, 2, "Gucci-World's Most Luxurious Fashion Brand"),
    (78708, 6, "Microsoft-Be What's Next"),
    (78709, 5, "Nike-Just Do It"),
    (78710, 3, "Trader Joe's-Your Neighborhood Grocery Store"),
    (78711, 1, "Starbucks-Coffee Love"),
    (78712, 2, "Home Depot-More Saving. More Doing."),
    (78713, 3, "Pep Boys-We go further to help you go farther"),
    (78714, 4, "Cinemark-The Best Seat In Town"),
    (78715, 5, "Domino's-Pizza Delivered Quickly"),
    (90210, 1, "Burger King-Home of the Whopper"),
    (60611, 2, "Prada-Luxury Fashion Brand"),
    (94103, 3, "Tesla-Drive Electric"),
    (10019, 4, "Adidas-Impossible is Nothing"),
    (90046, 5, "Gap-Individuals who embrace the spirit of individuality"),
    (90210, 6, "Panda Express-American Chinese Cuisine"),
    (10011, 7, "Samsung-Inspire the World, Create the Future"),
    (60654, 8, "Mobil-One-Engineered for Performance"),
    (94123, 9, "Costco-It's a Membership Worth Having"),
    (78701, 10, "Baskin Robbins-More Flavors. More Fun."),
    (78702, 1, "Verizon-Can you hear me now? Good."),
    (78703, 2, "Lush-Fresh Handmade Cosmetics"),
    (78704, 3, "Cheesecake Factory-A Rich Experience"),
    (78705, 4, "Abercrombie & Fitch-Casual Luxury"),
    (78706, 5, "McAlister's Deli-Handcrafted Sandwiches"),
    (78707, 6, "KFC-Finger Lickin' Good"),
    (78708, 7, "Dunkin'-America Runs on Dunkin'"),
    (78709, 8, "Hollister Co.-California Style"),
    (78710, 9, "Bath & Body Works-Love and Sunshine"),
    (78711, 10, "AT&T-Mobilizing Your World"),
    (78712, 1, "Macy's-Fashion. Family. Foundation."),
    (78713, 2, "Pizza Hut-Make it Great"),
    (78714, 3, "Gap Kids-Let's Kids Be Kids"),
    (78715, 4, "T.J. Maxx-Maxx Life"),
    (90210, 5, "Jimmy John's-Freaky Fast. Freaky Fresh."),
    (60611, 6, "Victoria's Secret-A World of Seduction"),
    (94103, 7, "Sephora-Beauty Together"),
    (10019, 8, "Burger King-Be Your Way"),
    (90046, 9, "Old Navy-Style for Everyone"),
    (90210, 10, "Starbucks-Bring Your Love of Coffee to Life"),
    (10011, 1, "CVS Pharmacy-Health is Everything"),
    (60654, 2, "Sbarro-Eat like an Italian"),
    (94123, 3, "Forever 21-Fashion at your fingertips"),
    (78701, 4, "T-Mobile-America's Largest 5G Network"),
    (78702, 5, "Panera Bread-Food as it Should Be"),
    (78703, 6, "Chili's-Every Guest Counts"),
    (78704, 7, "Burger King-Your Way, Way Better"),
    (78705, 8, "H&M-Fashion and Quality at the Best Price"),
    (78706, 9, "Popeyes-Louisiana Kitchen"),
    (78707, 10, "Dairy Queen-Fan Food, Not Fast Food"),
    (78708, 1, "Dillard's-The Style of Your Life"),
    (78709, 2, "Buffalo Wild Wings-Wings. Beer. Sports."),
    (78710, 3, "Marshalls-Your Surprise Is Waiting"),
    (78711, 4, "Forever 21-For the Next Generation"),
    (78712, 5, "Tim Hortons-Always Fresh"),
    (78713, 6, "Raising Cane's-One Love"),
    (78714, 7, "PetSmart-We Love Pets"),
    (78717, 1, "Five Guys-Burgers and Fries"),
    (90210, 9, "Chase Bank-Make More of What's Yours"),
    (60611, 10, "T.J. Maxx-Maxx Life"),
    (94103, 1, "Bed Bath & Beyond-Go Beyond"),
    (10019, 2, "Chase Bank-Make More of What's Yours"),
    (90046, 3, "Sonic Drive-In-America's Drive-In"),
    (90210, 4, "Trader Joe's-Your Neighborhood Grocery Store"),
    (10011, 5, "Jamba Juice-Smoothie Love"),
    (60654, 6, "Walgreens-Trusted Since 1901"),
    (94123, 7, "Foot Locker-Sneakers and Apparel"),
    (78701, 8, "Famous Footwear-Make Today Famous"),
    (78702, 9, "Lowe's-Do it Right for Less"),
    (78703, 10, "Denny's-America's Diner is Always Open"),
    (78704, 1, "Jack in the Box-We Don't Make It Until You Order It"),
    (78705, 2, "Raising Cane's-One Love"),
    (78706, 1, "Sonic Drive-In-America's Drive-In"),
    (78707, 4, "Staples-That was Easy"),
    (78708, 5, "Buffalo Wild Wings-Wings. Beer. Sports."),
    (78709, 6, "Whataburger-Just Like You Like It"),
    (78710, 7, "CVS Pharmacy-Health is Everything"),
    (78711, 8, "Subway-Fresh Fit For Kids"),
    (78712, 9, "Chase Bank-Make More of What's Yours"),
    (78713, 10, "Walgreens-Trusted Since 1901"),
    (78714, 1, "Dollar General-Save Time. Save Money. Every Day!"),
    (78715, 2, "Taco Bell-Best Food"),
    (90210, 3, "McDonald's-World's best Burger"),
    (60611, 4, "Chick-fil-A-Eat Mor Chikin"),
    (94103, 5, "Walmart-Price at lowest")
]

In [6]:
#Merchants sending advertisement to Alice
from binascii import hexlify, unhexlify
AliceZipCode=78717
AliceCategory=1
lstAliceAdd=[]
lstAliceNotAdd=[]

for item in Merchants:
    MerchantZipCode, MerchantCategory, MerchantAd = item
    message_int = string_to_int(MerchantAd)
   
    #Calculate Zipcode proximity between Alice And Merchant
    ZipProximity=CalculateProximity(Alice_public_key,Alice_private_key,AliceZipCode,MerchantZipCode)
    
    #Calculate Category proximity between Alice And Merchant
    CategoryProximity=CalculateProximity(Alice_public_key,Alice_private_key,AliceCategory,MerchantCategory)

    #Encrypt the Merchant Advertisement
    EncryptedAd=encrypt(Alice_public_key, message_int)
    Rvalue,CipherValue=EncryptedAd

    #If Not same  Zip and Category betwen Alice and Merchant  discard proximity with XNOR function
    ZipProximityXNOR=xnor(ZipProximity,1)
    CategoryProximityXNOR=xnor(CategoryProximity,1)
    
    #Multiply proximity with Encrypted Cipher value. 
    #This will discard all encrypted Advertisement that are not proximate with Zip and Category
    CipherValue=int(CipherValue*ZipProximityXNOR*CategoryProximityXNOR)

    #decrypt advertisement that are only proximate with Zip and Category
    dm=decrypt(Alice_private_key, (Rvalue,CipherValue))
    x = format(dm, 'x')
    #If Not same Zip and Category Ciper Value will be 0 after multiplying with XNOR function
    if(CipherValue==0):
        lstAliceNotAdd.append(item)
        #print('You are not the recipient')
    else:
        decryptedAdd=unhexlify(x).decode("utf-8")
        lstAliceAdd.append(decryptedAdd)

print('Ad for Alice:')
for strAd in lstAliceAdd:
    print(strAd)
print('\nBelow are not selected and unknown to Alice:')
for strAd in lstAliceNotAdd:
    print(strAd)

Ad for Alice:
Taco Bell-Best Food
Chick-fil-A-Eat Mor Chikin
Five Guys-Burgers and Fries

Below are not selected and unknown to Alice:
(78717, 4, "McDonald's-World's best Burger")
(78717, 2, 'Walmart-Price at lowest')
(78717, 9, 'Sushi Bar-Best Sushi In Town!')
(90210, 1, 'Chipotle-Mexican Grill')
(10001, 2, 'Target-Expect More, Pay Less')
(60611, 3, 'Jiffy Lube-Leave Worry Behind')
(94103, 4, 'AMC Theatres-Amazing Cinematic Experience')
(10019, 1, 'Shake Shack-Modern Day Roadside Burger Stand')
(90046, 9, 'Nobu-Malibu, CA')
(90210, 5, 'Louis Vuitton-Timeless Elegance')
(10011, 6, 'Apple-Think Different')
(60654, 7, 'Lululemon-Yoga-inspired athletic wear')
(94123, 8, "Whole Foods Market-America's Healthiest Grocery Store")
(78702, 2, 'Best Buy-Expert Service. Unbeatable Price.')
(78703, 3, 'Firestone Complete Auto Care-Keep Your Car Running Newer, Longer')
(78704, 4, 'Alamo Drafthouse Cinema-Experience the Difference')
(78705, 1, 'Subway-Fresh Fit For Kids')
(78706, 9, "PF Chang's-Chin